# 200520
OpenPose

1. OpenCV 설치
    >pip install opencv-python
2. MPI 및 COCO 모델 다운로드 후 model/로 이동, coco.caffemodel  coco.prototxt  mpi.caffemodel  mpi.prototxt  test.md
##### MPII Model(caffe/prototxt)
    http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/mpi/pose_iter_160000.caffemodel
    
    이름 변경!! pose_iter_160000.caffemodel -> mpi.caffemodel
    http://blogattach.naver.net/bf2aa3196d5a318ea4442f1a22c4b5c163344f/20200220_77_blogfile/shino1025_1582181201869_SH72yM_prototxt/mpi.prototxt
##### COCO Model(caffe/prototxt)
    http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/coco/pose_iter_440000.caffemodel

    이름 변경!! pose_iter_440000.caffemodel -> coco.caffemodel
   http://blogattach.naver.net/ac39b00016272690b657360d34d7addf7222d9dc/20200220_183_blogfile/shino1025_1582181217717_FTsb78_prototxt/coco.prototxt
    

3. 이미지/영상 탐지 - output/에 결과 생성
    > python image.py

    > python video.py
    
    
    
> 200MB 모델 수행시 1~1.5초 소요, CPU가 더 빠르다

In [1]:
##
import cv2
import time
import numpy as np
import os

USE_GPU = False # False

def proc(module, video):
    
    if module == "mpi":
        protoFile   = "./model/mpi.prototxt"
        weightsFile = "./model/mpi.caffemodel"
        nPoints = 15  # data에 있는 point 갯수
        POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13]]
    else:
        protoFile   = "./model/coco.prototxt"
        weightsFile = "./model/coco.caffemodel"
        nPoints = 18
        POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

    inWidth  = 368
    inHeight = 368
    threshold= 0.1
    input_source = 'input/' + video
    print(input_source)
    
    ## video 읽어오기
    cap = cv2.VideoCapture(input_source)
    hasFrame, frame = cap.read()
    # video 저장용, for avi
    vid_writer = cv2.VideoWriter('./output/' + video.split(".")[0] + "_"+ module + '.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))
    
    # caffe model을 opencv에서 읽어오기
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
    while cv2.waitKey(1) < 0:
        t = time.time()
        hasFrame, frame = cap.read()  # frame 읽기
        frameCopy = np.copy(frame)
        if not hasFrame:
            cv2.waitKey()
            break
        frameWidth  = frame.shape[1]
        frameHeight = frame.shape[0]
        # 입력 형태로 이미지 변환
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                          (0, 0, 0), swapRB=False, crop=False)
        net.setInput(inpBlob)
        output = net.forward()  # 추론!
        
        H = output.shape[2]
        W = output.shape[3]
        points = []
        for i in range(nPoints):
            probMap = output[0, i, :, :]
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H
            if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                points.append((int(x), int(y)))
            else :
                points.append(None)
        for pair in POSE_PAIRS:
            partA = pair[0]
            partB = pair[1]
            if points[partA] and points[partB]:
                cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
                cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(frame, "Test Time = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
        vid_writer.write(frame)  # 영상 저장
    vid_writer.release()




In [ ]:
if __name__ == '__main__':
    module = ['mpi', 'coco']
    for i in module:
        for j in os.listdir('input'):
            if j[-3:] in ['mp4','avi']:
                print(i, j)
                proc(i, j)

In [ ]:
proc('mpi', 'tennis')

In [4]:
module = 'mpi'
video  ='rain.avi'

In [5]:
input_source = 'input/' + video
print(module, ' : ', input_source)

if module == "mpi":
    protoFile   = "./model/mpi.prototxt"
    weightsFile = "./model/mpi.caffemodel"
    nPoints = 15  # data에 있는 point 갯수
    POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13]]
else:
    protoFile   = "./model/coco.prototxt"
    weightsFile = "./model/coco.caffemodel"
    nPoints = 18
    POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

inWidth  = 368
inHeight = 368
threshold= 0.1


## video 읽어오기
cap = cv2.VideoCapture(input_source)
hasFrame, frame = cap.read()
# video 저장용, for avi
vid_writer = cv2.VideoWriter('./output/' + video.split(".")[0] + "_"+ module + '.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))

# caffe model을 opencv에서 읽어오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
while cv2.waitKey(1) < 0:
    t = time.time()
    hasFrame, frame = cap.read()  # frame 읽기
    frameCopy = np.copy(frame)
    if not hasFrame:
        cv2.waitKey()
        break
    frameWidth  = frame.shape[1]
    frameHeight = frame.shape[0]
    # 입력 형태로 이미지 변환
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                      (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()  # 추론!

    H = output.shape[2]
    W = output.shape[3]
    points = []
    for i in range(nPoints):
        probMap = output[0, i, :, :]
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H
        if prob > threshold : 
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
        else :
            points.append(None)
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
    cv2.putText(frame, "Test Time = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    vid_writer.write(frame)  # 영상 저장
vid_writer.release()

print('done')

mpi  :  input/rain.avi


In [6]:
print('done')

done


In [ ]:
import torch
model = torch.nn.LSTM(256, 2, 1, bidirectional=False)
model.eval()

dummy_input = torch.randn(1, 37, 256)
torch.onnx.export(model, dummy_input, "crnn.onnx", verbose=True)

import cv2
net=cv2.dnn.readNet("crnn.onnx")
net=cv2.dnn.readNet("crnn.onnx")